In [90]:
import pandas as pd
import numpy as np
import re


In [91]:
def save_grouped_data_to_csv(df, group_column, columns_not_to_keep, position_level):
    grouped_df = df.groupby(group_column)
    
    for name, group in grouped_df:
        print("name : ", name)
        filtered_df = group.drop(columns=columns_not_to_keep)
        first_column_name = filtered_df.columns[0].lower()

        sanitized_name = name.replace(" ", "-").replace("/", "-").replace(",", "")
        sanitized_name = re.sub(r'-+', '-', sanitized_name)  # Replace multiple hyphens with a single one
        sanitized_name = sanitized_name.strip('-')  # Remove leading and trailing hyphens
        
        file_path = f'../created_csv/result/{position_level}/{sanitized_name}_{first_column_name}.csv'
        
        filtered_df.to_csv(file_path, index=False)

### 사용할 Job List

In [92]:
filtered_job_list = [
'BlockChain',
'Developer, full-stack',
'Developer, back-end',
'Developer, front-end',
'Developer, game or graphics',
'Developer, mobile',
'UX / UI Designer',
'Engineer, data',
'Database administrator',
'Data scientist or machine learning specialist',
'Data or business analyst',
'Product manager',
'Developer Advocate',
'Developer, QA or test',
'DevOps specialist'
]

### Language

In [93]:
junior_devtype_language_df = pd.read_csv("../stak_overflow_processing/junior_devtype_language.csv")
middle_devtype_language_df = pd.read_csv("../stak_overflow_processing/middle_devtype_language.csv")
senior_devtype_language_df = pd.read_csv("../stak_overflow_processing/senior_devtype_language.csv")


In [94]:
junior_devtype_language_df.head(5)

,YearsGroup,DevType,Language,Count,Rank
0,junior,Academic researcher,Python,54,1
1,junior,Academic researcher,JavaScript,44,2
2,junior,Academic researcher,SQL,40,3
3,junior,Academic researcher,HTML/CSS,40,4
4,junior,Academic researcher,Java,24,5


In [95]:
middle_devtype_language_df.head(5)

,YearsGroup,DevType,Language,Count,Rank
0,middle,Academic researcher,Python,257,1
1,middle,Academic researcher,JavaScript,243,2
2,middle,Academic researcher,HTML/CSS,228,3
3,middle,Academic researcher,SQL,205,4
4,middle,Academic researcher,Java,149,5


In [96]:
senior_devtype_language_df.head(5)

,YearsGroup,DevType,Language,Count,Rank
0,senior,Academic researcher,Python,1454,1
1,senior,Academic researcher,JavaScript,1337,2
2,senior,Academic researcher,HTML/CSS,1235,3
3,senior,Academic researcher,SQL,1184,4
4,senior,Academic researcher,Java,833,5


In [97]:
# 정의한 직업 리스트에 속하는 자료만 filter
junior_filtered_devtype_language_df = junior_devtype_language_df[junior_devtype_language_df['DevType'].isin(filtered_job_list)]
middle_filtered_devtype_language_df = middle_devtype_language_df[middle_devtype_language_df['DevType'].isin(filtered_job_list)]
senior_filtered_devtype_language_df = senior_devtype_language_df[senior_devtype_language_df['DevType'].isin(filtered_job_list)]


In [98]:
# unique_langugaes
junior_unique_languages = junior_filtered_devtype_language_df['Language'].unique()
middle_unique_languages = middle_filtered_devtype_language_df['Language'].unique()
senior_unique_languages = senior_filtered_devtype_language_df['Language'].unique()


#### junior

In [99]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_primary_proglang_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_primary_proglang_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_primary_proglang_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_primary_proglang_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_primary_proglang_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_primary_proglang_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_primary_proglang_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_primary_proglang_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_primary_proglang_percentages_df.csv')

In [128]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_primary_proglang_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_primary_proglang_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_primary_proglang_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_primary_proglang_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_primary_proglang_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_primary_proglang_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_primary_proglang_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_primary_proglang_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_primary_proglang_percentages_df.csv')

In [130]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_primary_proglang_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_primary_proglang_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_primary_proglang_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_primary_proglang_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_primary_proglang_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_primary_proglang_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_primary_proglang_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_primary_proglang_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_primary_proglang_percentages_df.csv')

In [100]:
def update_counts(filtered_df, merge_df, filter_col, merge_col, devtype_col, count_col='Count'):
    for i, row in filtered_df.iterrows():
        value = row[filter_col]
        devtype = row['DevType']
        if devtype == devtype_col and value in merge_df[merge_col].values:
            additional_count = merge_df[merge_df[merge_col] == value][count_col].values[0]
            filtered_df.at[i, count_col] += additional_count
    return filtered_df


In [101]:
update_counts(junior_filtered_devtype_language_df, junior_data_business_analyst_df, 'Language', 'primary_proglang', 'Data or business analyst', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_database_administrator_df, 'Language', 'primary_proglang', 'Database administrator', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_developer_advocate_df, 'Language', 'primary_proglang', 'Developer Advocate', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_Developer_back_end_df, 'Language', 'primary_proglang', 'Developer, back-end', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_Developer_front_end_df, 'Language', 'primary_proglang', 'Developer, front-end', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_Developer_full_stack_df, 'Language', 'primary_proglang', 'Developer, full-stack', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_Developer_mobile_df, 'Language', 'primary_proglang', 'Developer, mobile', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_DevOps_specialist_df, 'Language', 'primary_proglang', 'DevOps specialist', 'Count')
update_counts(junior_filtered_devtype_language_df, junior_UX_UI_Designer_df, 'Language', 'primary_proglang', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Language,Count,Rank
85,junior,Data or business analyst,SQL,230,1
86,junior,Data or business analyst,Python,224,2
87,junior,Data or business analyst,HTML/CSS,128,3
88,junior,Data or business analyst,JavaScript,127,4
89,junior,Data or business analyst,R,74,5
...,...,...,...,...,...
876,junior,Product manager,Delphi,1,30
877,junior,Product manager,Scala,1,31
878,junior,Product manager,Perl,1,32
879,junior,Product manager,Matlab,1,33


In [132]:
update_counts(middle_filtered_devtype_language_df, middle_data_business_analyst_df, 'Language', 'primary_proglang', 'Data or business analyst', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_database_administrator_df, 'Language', 'primary_proglang', 'Database administrator', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_developer_advocate_df, 'Language', 'primary_proglang', 'Developer Advocate', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_Developer_back_end_df, 'Language', 'primary_proglang', 'Developer, back-end', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_Developer_front_end_df, 'Language', 'primary_proglang', 'Developer, front-end', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_Developer_full_stack_df, 'Language', 'primary_proglang', 'Developer, full-stack', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_Developer_mobile_df, 'Language', 'primary_proglang', 'Developer, mobile', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_DevOps_specialist_df, 'Language', 'primary_proglang', 'DevOps specialist', 'Count')
update_counts(middle_filtered_devtype_language_df, middle_UX_UI_Designer_df, 'Language', 'primary_proglang', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Language,Count,Rank
130,middle,Data or business analyst,Python,531,1
131,middle,Data or business analyst,SQL,517,2
132,middle,Data or business analyst,JavaScript,376,3
133,middle,Data or business analyst,HTML/CSS,369,4
134,middle,Data or business analyst,R,185,5
...,...,...,...,...,...
1122,middle,Product manager,Elixir,2,35
1123,middle,Product manager,Perl,2,36
1124,middle,Product manager,Apex,1,37
1125,middle,Product manager,COBOL,1,38


In [131]:
update_counts(senior_filtered_devtype_language_df, senior_data_business_analyst_df, 'Language', 'primary_proglang', 'Data or business analyst', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_database_administrator_df, 'Language', 'primary_proglang', 'Database administrator', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_developer_advocate_df, 'Language', 'primary_proglang', 'Developer Advocate', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_Developer_back_end_df, 'Language', 'primary_proglang', 'Developer, back-end', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_Developer_front_end_df, 'Language', 'primary_proglang', 'Developer, front-end', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_Developer_full_stack_df, 'Language', 'primary_proglang', 'Developer, full-stack', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_Developer_mobile_df, 'Language', 'primary_proglang', 'Developer, mobile', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_DevOps_specialist_df, 'Language', 'primary_proglang', 'DevOps specialist', 'Count')
update_counts(senior_filtered_devtype_language_df, senior_UX_UI_Designer_df, 'Language', 'primary_proglang', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Language,Count,Rank
153,senior,Data or business analyst,SQL,2974,1
154,senior,Data or business analyst,JavaScript,2408,2
155,senior,Data or business analyst,HTML/CSS,2213,3
156,senior,Data or business analyst,Python,2149,4
157,senior,Data or business analyst,C#,1302,5
...,...,...,...,...,...
1361,senior,Product manager,WebAssembly,4,49
1362,senior,Product manager,Apex,3,50
1363,senior,Product manager,Lisp,2,51
1364,senior,Product manager,Zig,2,52


In [134]:
save_grouped_data_to_csv(junior_filtered_devtype_language_df, 'DevType', ['DevType', 'YearsGroup'], 'junior')
save_grouped_data_to_csv(middle_filtered_devtype_language_df, 'DevType', ['DevType', 'YearsGroup'], 'middle')
save_grouped_data_to_csv(senior_filtered_devtype_language_df, 'DevType', ['DevType', 'YearsGroup'], 'senior')

name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name 

### Database

In [105]:
junior_devtype_database_df = pd.read_csv("../stak_overflow_processing/junior_devtype_database.csv")
middle_devtype_database_df = pd.read_csv("../stak_overflow_processing/middle_devtype_database.csv")
senior_devtype_database_df = pd.read_csv("../stak_overflow_processing/senior_devtype_database.csv")

# 정의한 직업 리스트에 속하는 자료만 filter
junior_filtered_devtype_database_df = junior_devtype_database_df[junior_devtype_database_df['DevType'].isin(filtered_job_list)]
middle_filtered_devtype_database_df = middle_devtype_database_df[middle_devtype_database_df['DevType'].isin(filtered_job_list)]
senior_filtered_devtype_database_df = senior_devtype_database_df[senior_devtype_database_df['DevType'].isin(filtered_job_list)]

In [106]:
junior_filtered_devtype_database_df

,YearsGroup,DevType,Database,Count,Rank
41,junior,Data or business analyst,MySQL,146,1
42,junior,Data or business analyst,PostgreSQL,131,2
43,junior,Data or business analyst,Microsoft SQL Server,105,3
44,junior,Data or business analyst,SQLite,101,4
45,junior,Data or business analyst,MongoDB,72,5
...,...,...,...,...,...
474,junior,Product manager,Dynamodb,1,18
475,junior,Product manager,Cosmos DB,1,19
476,junior,Product manager,Couchbase,1,20
477,junior,Product manager,BigQuery,1,21


In [107]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_db_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_db_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_db_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_db_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_db_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_db_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_db_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_db_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_db_percentages_df.csv')

In [135]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_db_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_db_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_db_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_db_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_db_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_db_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_db_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_db_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_db_percentages_df.csv')

In [136]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_db_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_db_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_db_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_db_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_db_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_db_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_db_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_db_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_db_percentages_df.csv')

In [108]:
update_counts(junior_filtered_devtype_database_df, junior_data_business_analyst_df, 'Database', 'db', 'Data or business analyst', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_database_administrator_df, 'Database', 'db', 'Database administrator', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_developer_advocate_df, 'Database', 'db', 'Developer Advocate', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_Developer_back_end_df, 'Database', 'db', 'Developer, back-end', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_Developer_front_end_df, 'Database', 'db', 'Developer, front-end', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_Developer_full_stack_df, 'Database', 'db', 'Developer, full-stack', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_Developer_mobile_df, 'Database', 'db', 'Developer, mobile', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_DevOps_specialist_df, 'Database', 'db', 'DevOps specialist', 'Count')
update_counts(junior_filtered_devtype_database_df, junior_UX_UI_Designer_df, 'Database', 'db', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Database,Count,Rank
41,junior,Data or business analyst,MySQL,147,1
42,junior,Data or business analyst,PostgreSQL,134,2
43,junior,Data or business analyst,Microsoft SQL Server,105,3
44,junior,Data or business analyst,SQLite,102,4
45,junior,Data or business analyst,MongoDB,73,5
...,...,...,...,...,...
474,junior,Product manager,Dynamodb,1,18
475,junior,Product manager,Cosmos DB,1,19
476,junior,Product manager,Couchbase,1,20
477,junior,Product manager,BigQuery,1,21


In [137]:
update_counts(middle_filtered_devtype_database_df, middle_data_business_analyst_df, 'Database', 'db', 'Data or business analyst', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_database_administrator_df, 'Database', 'db', 'Database administrator', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_developer_advocate_df, 'Database', 'db', 'Developer Advocate', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_Developer_back_end_df, 'Database', 'db', 'Developer, back-end', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_Developer_front_end_df, 'Database', 'db', 'Developer, front-end', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_Developer_full_stack_df, 'Database', 'db', 'Developer, full-stack', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_Developer_mobile_df, 'Database', 'db', 'Developer, mobile', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_DevOps_specialist_df, 'Database', 'db', 'DevOps specialist', 'Count')
update_counts(middle_filtered_devtype_database_df, middle_UX_UI_Designer_df, 'Database', 'db', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Database,Count,Rank
65,middle,Data or business analyst,MySQL,334,1
66,middle,Data or business analyst,PostgreSQL,319,2
67,middle,Data or business analyst,Microsoft SQL Server,296,3
68,middle,Data or business analyst,SQLite,237,4
69,middle,Data or business analyst,MongoDB,168,5
...,...,...,...,...,...
604,middle,Product manager,Snowflake,1,18
605,middle,Product manager,TiDB,1,19
606,middle,Product manager,InfluxDB,1,20
607,middle,Product manager,CouchDB,1,21


In [138]:
update_counts(senior_filtered_devtype_database_df, senior_data_business_analyst_df, 'Database', 'db', 'Data or business analyst', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_database_administrator_df, 'Database', 'db', 'Database administrator', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_developer_advocate_df, 'Database', 'db', 'Developer Advocate', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_Developer_back_end_df, 'Database', 'db', 'Developer, back-end', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_Developer_front_end_df, 'Database', 'db', 'Developer, front-end', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_Developer_full_stack_df, 'Database', 'db', 'Developer, full-stack', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_Developer_mobile_df, 'Database', 'db', 'Developer, mobile', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_DevOps_specialist_df, 'Database', 'db', 'DevOps specialist', 'Count')
update_counts(senior_filtered_devtype_database_df, senior_UX_UI_Designer_df, 'Database', 'db', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Database,Count,Rank
93,senior,Data or business analyst,MySQL,1885,1
94,senior,Data or business analyst,Microsoft SQL Server,1858,2
95,senior,Data or business analyst,PostgreSQL,1681,3
96,senior,Data or business analyst,SQLite,1270,4
97,senior,Data or business analyst,MongoDB,864,5
...,...,...,...,...,...
835,senior,Product manager,Cockroachdb,3,30
836,senior,Product manager,DuckDB,2,31
837,senior,Product manager,Datomic,2,32
838,senior,Product manager,Supabase,2,33


In [139]:
save_grouped_data_to_csv(junior_filtered_devtype_database_df, 'DevType', ['DevType', 'YearsGroup'], 'junior')
save_grouped_data_to_csv(middle_filtered_devtype_database_df, 'DevType', ['DevType', 'YearsGroup'], 'middle')
save_grouped_data_to_csv(senior_filtered_devtype_database_df, 'DevType', ['DevType', 'YearsGroup'], 'senior')

name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name 

### Framework

In [110]:
junior_devtype_framework_df = pd.read_csv("../stak_overflow_processing/junior_devtype_framework.csv")
middle_devtype_framework_df = pd.read_csv("../stak_overflow_processing/middle_devtype_framework.csv")
senior_devtype_framework_df = pd.read_csv("../stak_overflow_processing/senior_devtype_framework.csv")

# 정의한 직업 리스트에 속하는 자료만 filter
junior_filtered_devtype_framework_df = junior_devtype_framework_df[junior_devtype_framework_df['DevType'].isin(filtered_job_list)]
middle_filtered_devtype_framework_df = middle_devtype_framework_df[middle_devtype_framework_df['DevType'].isin(filtered_job_list)]
senior_filtered_devtype_framework_df = senior_devtype_framework_df[senior_devtype_framework_df['DevType'].isin(filtered_job_list)]

junior_filtered_devtype_framework_df

,YearsGroup,DevType,Webframe,Count,Rank
54,junior,Data or business analyst,Django,62,1
55,junior,Data or business analyst,Flask,47,2
56,junior,Data or business analyst,React.js,43,3
57,junior,Data or business analyst,jQuery,42,4
58,junior,Data or business analyst,Express,27,5
...,...,...,...,...,...
607,junior,Product manager,Symfony,1,22
608,junior,Product manager,Svelte,1,23
609,junior,Product manager,Fastify,1,24
610,junior,Product manager,ASP.NET Core,1,25


In [111]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_ide_main_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_ide_main_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_ide_main_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_ide_main_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_ide_main_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_ide_main_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_ide_main_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_ide_main_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_ide_main_percentages_df.csv')

In [140]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_ide_main_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_ide_main_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_ide_main_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_ide_main_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_ide_main_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_ide_main_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_ide_main_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_ide_main_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_ide_main_percentages_df.csv')

In [141]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_ide_main_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_ide_main_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_ide_main_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_ide_main_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_ide_main_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_ide_main_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_ide_main_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_ide_main_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_ide_main_percentages_df.csv')

In [112]:
junior_Developer_back_end_df.head(5)

,Unnamed: 0,ide_main,Percentage,Count
0,0,IntelliJ IDEA,40.0,2
1,1,VS Code (Visual Studio Code),40.0,2
2,2,Emacs,20.0,1


In [113]:
update_counts(junior_filtered_devtype_framework_df, junior_data_business_analyst_df, 'Webframe', 'ide_main', 'Data or business analyst', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_database_administrator_df, 'Webframe', 'ide_main', 'Database administrator', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_developer_advocate_df, 'Webframe', 'ide_main', 'Developer Advocate', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_Developer_back_end_df, 'Webframe', 'ide_main', 'Developer, back-end', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_Developer_front_end_df, 'Webframe', 'ide_main', 'Developer, front-end', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_Developer_full_stack_df, 'Webframe', 'ide_main', 'Developer, full-stack', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_Developer_mobile_df, 'Webframe', 'ide_main', 'Developer, mobile', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_DevOps_specialist_df, 'Webframe', 'ide_main', 'DevOps specialist', 'Count')
update_counts(junior_filtered_devtype_framework_df, junior_UX_UI_Designer_df, 'Webframe', 'ide_main', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Webframe,Count,Rank
54,junior,Data or business analyst,Django,62,1
55,junior,Data or business analyst,Flask,47,2
56,junior,Data or business analyst,React.js,43,3
57,junior,Data or business analyst,jQuery,42,4
58,junior,Data or business analyst,Express,27,5
...,...,...,...,...,...
607,junior,Product manager,Symfony,1,22
608,junior,Product manager,Svelte,1,23
609,junior,Product manager,Fastify,1,24
610,junior,Product manager,ASP.NET Core,1,25


In [142]:
update_counts(middle_filtered_devtype_framework_df, middle_data_business_analyst_df, 'Webframe', 'ide_main', 'Data or business analyst', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_database_administrator_df, 'Webframe', 'ide_main', 'Database administrator', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_developer_advocate_df, 'Webframe', 'ide_main', 'Developer Advocate', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_Developer_back_end_df, 'Webframe', 'ide_main', 'Developer, back-end', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_Developer_front_end_df, 'Webframe', 'ide_main', 'Developer, front-end', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_Developer_full_stack_df, 'Webframe', 'ide_main', 'Developer, full-stack', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_Developer_mobile_df, 'Webframe', 'ide_main', 'Developer, mobile', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_DevOps_specialist_df, 'Webframe', 'ide_main', 'DevOps specialist', 'Count')
update_counts(middle_filtered_devtype_framework_df, middle_UX_UI_Designer_df, 'Webframe', 'ide_main', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Webframe,Count,Rank
93,middle,Data or business analyst,Flask,167,1
94,middle,Data or business analyst,React.js,157,2
95,middle,Data or business analyst,jQuery,156,3
96,middle,Data or business analyst,Django,154,4
97,middle,Data or business analyst,Express,89,5
...,...,...,...,...,...
764,middle,Product manager,Drupal,4,21
765,middle,Product manager,Fastify,2,22
766,middle,Product manager,Nuxt.js,2,23
767,middle,Product manager,ASP.NET CORE,1,24


In [143]:
update_counts(senior_filtered_devtype_framework_df, senior_data_business_analyst_df, 'Webframe', 'ide_main', 'Data or business analyst', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_database_administrator_df, 'Webframe', 'ide_main', 'Database administrator', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_developer_advocate_df, 'Webframe', 'ide_main', 'Developer Advocate', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_Developer_back_end_df, 'Webframe', 'ide_main', 'Developer, back-end', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_Developer_front_end_df, 'Webframe', 'ide_main', 'Developer, front-end', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_Developer_full_stack_df, 'Webframe', 'ide_main', 'Developer, full-stack', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_Developer_mobile_df, 'Webframe', 'ide_main', 'Developer, mobile', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_DevOps_specialist_df, 'Webframe', 'ide_main', 'DevOps specialist', 'Count')
update_counts(senior_filtered_devtype_framework_df, senior_UX_UI_Designer_df, 'Webframe', 'ide_main', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Webframe,Count,Rank
106,senior,Data or business analyst,jQuery,1169,1
107,senior,Data or business analyst,React.js,766,2
108,senior,Data or business analyst,ASP.NET,712,3
109,senior,Data or business analyst,Angular,600,4
110,senior,Data or business analyst,Flask,574,5
...,...,...,...,...,...
917,senior,Product manager,CodeIgniter,4,33
918,senior,Product manager,NestJS,4,34
919,senior,Product manager,Elm,3,35
920,senior,Product manager,Remix,2,36


In [144]:
save_grouped_data_to_csv(junior_filtered_devtype_framework_df, 'DevType', ['DevType', 'YearsGroup'], 'junior')
save_grouped_data_to_csv(middle_filtered_devtype_framework_df, 'DevType', ['DevType', 'YearsGroup'], 'middle')
save_grouped_data_to_csv(senior_filtered_devtype_framework_df, 'DevType', ['DevType', 'YearsGroup'], 'senior')

name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name 

### Idle

In [145]:
junior_devtype_tool_df = pd.read_csv("../stak_overflow_processing/junior_devtype_tools.csv")
middle_devtype_tool_df = pd.read_csv("../stak_overflow_processing/middle_devtype_tools.csv")
senior_devtype_tool_df = pd.read_csv("../stak_overflow_processing/senior_devtype_tools.csv")

# 정의한 직업 리스트에 속하는 자료만 filter
junior_filtered_devtype_tool_df = junior_devtype_tool_df[junior_devtype_tool_df['DevType'].isin(filtered_job_list)]
middle_filtered_devtype_tool_df = middle_devtype_tool_df[middle_devtype_tool_df['DevType'].isin(filtered_job_list)]
senior_filtered_devtype_tool_df = senior_devtype_tool_df[senior_devtype_tool_df['DevType'].isin(filtered_job_list)]

junior_filtered_devtype_tool_df

,YearsGroup,DevType,Tools,Count,Rank
68,junior,Data or business analyst,Visual Studio Code,82,1
69,junior,Data or business analyst,IPython/Jupyter,45,2
70,junior,Data or business analyst,Github,42,3
71,junior,Data or business analyst,Notepad++,32,4
72,junior,Data or business analyst,PyCharm,30,5
...,...,...,...,...,...
837,junior,Product manager,IPython,1,32
838,junior,Product manager,"RAD Studio (Delphi, C++ Builder)",1,33
839,junior,Product manager,NetBeans,1,34
840,junior,Product manager,PHPStorm,1,35


In [146]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_ide_main_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_ide_main_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_ide_main_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_ide_main_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_ide_main_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_ide_main_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_ide_main_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_ide_main_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_ide_main_percentages_df.csv')

In [147]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_ide_main_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_ide_main_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_ide_main_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_ide_main_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_ide_main_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_ide_main_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_ide_main_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_ide_main_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_ide_main_percentages_df.csv')

In [148]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_ide_main_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_ide_main_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_ide_main_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_ide_main_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_ide_main_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_ide_main_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_ide_main_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_ide_main_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_ide_main_percentages_df.csv')

In [149]:
update_counts(junior_filtered_devtype_tool_df, junior_data_business_analyst_df, 'Tools', 'ide_main', 'Data or business analyst', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_database_administrator_df, 'Tools', 'ide_main', 'Database administrator', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_developer_advocate_df, 'Tools', 'ide_main', 'Developer Advocate', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_Developer_back_end_df, 'Tools', 'ide_main', 'Developer, back-end', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_Developer_front_end_df, 'Tools', 'ide_main', 'Developer, front-end', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_Developer_full_stack_df, 'Tools', 'ide_main', 'Developer, full-stack', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_Developer_mobile_df, 'Tools', 'ide_main', 'Developer, mobile', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_DevOps_specialist_df, 'Tools', 'ide_main', 'DevOps specialist', 'Count')
update_counts(junior_filtered_devtype_tool_df, junior_UX_UI_Designer_df, 'Tools', 'ide_main', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Tools,Count,Rank
68,junior,Data or business analyst,Visual Studio Code,82,1
69,junior,Data or business analyst,IPython/Jupyter,45,2
70,junior,Data or business analyst,Github,42,3
71,junior,Data or business analyst,Notepad++,32,4
72,junior,Data or business analyst,PyCharm,31,5
...,...,...,...,...,...
837,junior,Product manager,IPython,1,32
838,junior,Product manager,"RAD Studio (Delphi, C++ Builder)",1,33
839,junior,Product manager,NetBeans,1,34
840,junior,Product manager,PHPStorm,1,35


In [150]:
update_counts(middle_filtered_devtype_tool_df, middle_data_business_analyst_df, 'Tools', 'ide_main', 'Data or business analyst', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_database_administrator_df, 'Tools', 'ide_main', 'Database administrator', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_developer_advocate_df, 'Tools', 'ide_main', 'Developer Advocate', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_Developer_back_end_df, 'Tools', 'ide_main', 'Developer, back-end', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_Developer_front_end_df, 'Tools', 'ide_main', 'Developer, front-end', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_Developer_full_stack_df, 'Tools', 'ide_main', 'Developer, full-stack', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_Developer_mobile_df, 'Tools', 'ide_main', 'Developer, mobile', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_DevOps_specialist_df, 'Tools', 'ide_main', 'DevOps specialist', 'Count')
update_counts(middle_filtered_devtype_tool_df, middle_UX_UI_Designer_df, 'Tools', 'ide_main', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Tools,Count,Rank
116,middle,Data or business analyst,Visual Studio Code,238,1
117,middle,Data or business analyst,Github,150,2
118,middle,Data or business analyst,IPython/Jupyter,117,3
119,middle,Data or business analyst,PyCharm,96,4
120,middle,Data or business analyst,Slack,95,5
...,...,...,...,...,...
1053,middle,Product manager,Facebook Workplace,2,33
1054,middle,Product manager,DataGrip,1,34
1055,middle,Product manager,GoLand,1,35
1056,middle,Product manager,RubyMine,1,36


In [151]:
update_counts(senior_filtered_devtype_tool_df, senior_data_business_analyst_df, 'Tools', 'ide_main', 'Data or business analyst', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_database_administrator_df, 'Tools', 'ide_main', 'Database administrator', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_developer_advocate_df, 'Tools', 'ide_main', 'Developer Advocate', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_Developer_back_end_df, 'Tools', 'ide_main', 'Developer, back-end', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_Developer_front_end_df, 'Tools', 'ide_main', 'Developer, front-end', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_Developer_full_stack_df, 'Tools', 'ide_main', 'Developer, full-stack', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_Developer_mobile_df, 'Tools', 'ide_main', 'Developer, mobile', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_DevOps_specialist_df, 'Tools', 'ide_main', 'DevOps specialist', 'Count')
update_counts(senior_filtered_devtype_tool_df, senior_UX_UI_Designer_df, 'Tools', 'ide_main', 'UX / UI Designer', 'Count')

,YearsGroup,DevType,Tools,Count,Rank
134,senior,Data or business analyst,Visual Studio Code,1171,1
135,senior,Data or business analyst,Github,813,2
136,senior,Data or business analyst,Notepad++,670,3
137,senior,Data or business analyst,Visual Studio,641,4
138,senior,Data or business analyst,Slack,522,5
...,...,...,...,...,...
1277,senior,Product manager,Goland,2,51
1278,senior,Product manager,VSCodium,1,52
1279,senior,Product manager,Code::Blocks,1,53
1280,senior,Product manager,Micro,1,54


In [152]:
save_grouped_data_to_csv(junior_filtered_devtype_tool_df, 'DevType', ['DevType', 'YearsGroup'], 'junior')
save_grouped_data_to_csv(middle_filtered_devtype_tool_df, 'DevType', ['DevType', 'YearsGroup'], 'middle')
save_grouped_data_to_csv(senior_filtered_devtype_tool_df, 'DevType', ['DevType', 'YearsGroup'], 'senior')

name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name :  Developer, back-end
name :  Developer, front-end
name :  Developer, full-stack
name :  Developer, game or graphics
name :  Developer, mobile
name :  Engineer, data
name :  Product manager
name :  Data or business analyst
name :  Data scientist or machine learning specialist
name :  Database administrator
name :  DevOps specialist
name :  Developer Advocate
name :  Developer, QA or test
name 

In [119]:
def transform_and_save_data(df, position_level, file_name):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    transformed_df = df.drop(columns=['Percentage'])

    transformed_df['Rank'] = transformed_df['Count'].rank(method='max', ascending=False).astype(int)

    pattern = r'_(.*?)_'
    match = re.search(pattern, file_name)
    dev_name = match.group(1)

    first_column_name = transformed_df.columns[0].lower()
    sanitized_name = dev_name.replace(" ", "-").replace("/", "-").replace(",", "")
    sanitized_name = re.sub(r'-+', '-', sanitized_name)  # Replace multiple hyphens with a single one
    sanitized_name = sanitized_name.strip('-')  # Remove leading and trailing hyphens
     
    # file_path = f'../created_csv/result/{first_column_name}_tools.csv'
    file_path = f'../created_csv/result/{position_level}/{dev_name}_{first_column_name}.csv'
    transformed_df.to_csv(file_path, index=False)
    # transformed_df.to_csv(file_path, index=False)

## Job_code

In [120]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_job_code_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_job_code_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_job_code_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_job_code_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_job_code_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_job_code_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_job_code_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_job_code_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_job_code_percentages_df.csv')

In [153]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_job_code_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_job_code_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_job_code_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_job_code_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_job_code_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_job_code_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_job_code_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_job_code_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_job_code_percentages_df.csv')

In [154]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_job_code_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_job_code_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_job_code_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_job_code_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_job_code_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_job_code_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_job_code_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_job_code_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_job_code_percentages_df.csv')

In [121]:
transform_and_save_data(junior_data_business_analyst_df, 'junior', 'junior_Data-or-business-analyst_job_code_percentages_df')
transform_and_save_data(junior_database_administrator_df, 'junior', 'junior_Database-administrator_job_code_percentages_df')
transform_and_save_data(junior_developer_advocate_df, 'junior', 'junior_Developer-Advocate_job_code_percentages_df')
transform_and_save_data(junior_Developer_back_end_df, 'junior', 'junior_Developer-back-end_job_code_percentages_df')
transform_and_save_data(junior_Developer_front_end_df, 'junior', 'junior_Developer-front-end_job_code_percentages_df')
transform_and_save_data(junior_Developer_full_stack_df, 'junior', 'junior_Developer-full-stack_job_code_percentages_df')
transform_and_save_data(junior_Developer_mobile_df, 'junior', 'junior_Developer-mobile_job_code_percentages_df')
transform_and_save_data(junior_DevOps_specialist_df, 'junior', 'junior_DevOps-specialist_job_code_percentages_df')
transform_and_save_data(junior_UX_UI_Designer_df, 'junior', 'junior_UX-UI-Designer_job_code_percentages_df')

In [156]:
transform_and_save_data(middle_data_business_analyst_df, 'middle', 'middle_Data-or-business-analyst_job_code_percentages_df')
transform_and_save_data(middle_database_administrator_df, 'middle', 'middle_Database-administrator_job_code_percentages_df')
transform_and_save_data(middle_developer_advocate_df, 'middle', 'middle_Developer-Advocate_job_code_percentages_df')
transform_and_save_data(middle_Developer_back_end_df, 'middle', 'middle_Developer-back-end_job_code_percentages_df')
transform_and_save_data(middle_Developer_front_end_df, 'middle', 'middle_Developer-front-end_job_code_percentages_df')
transform_and_save_data(middle_Developer_full_stack_df, 'middle', 'middle_Developer-full-stack_job_code_percentages_df')
transform_and_save_data(middle_Developer_mobile_df, 'middle', 'middle_Developer-mobile_job_code_percentages_df')
transform_and_save_data(middle_DevOps_specialist_df, 'middle', 'middle_DevOps-specialist_job_code_percentages_df')
transform_and_save_data(middle_UX_UI_Designer_df, 'middle', 'middler_UX-UI-Designer_job_code_percentages_df')

In [157]:
transform_and_save_data(senior_data_business_analyst_df, 'senior', 'senior_Data-or-business-analyst_job_code_percentages_df')
transform_and_save_data(senior_database_administrator_df, 'senior', 'senior_Database-administrator_job_code_percentages_df')
transform_and_save_data(senior_developer_advocate_df, 'senior', 'senior_Developer-Advocate_job_code_percentages_df')
transform_and_save_data(senior_Developer_back_end_df, 'senior', 'senior_Developer-back-end_job_code_percentages_df')
transform_and_save_data(senior_Developer_front_end_df, 'senior', 'senior_Developer-front-end_job_code_percentages_df')
transform_and_save_data(senior_Developer_full_stack_df, 'senior', 'senior_Developer-full-stack_job_code_percentages_df')
transform_and_save_data(senior_Developer_mobile_df, 'senior', 'senior_Developer-mobile_job_code_percentages_df')
transform_and_save_data(senior_DevOps_specialist_df, 'senior', 'senior_DevOps-specialist_job_code_percentages_df')
transform_and_save_data(senior_UX_UI_Designer_df, 'senior', 'senior_UX-UI-Designer_job_code_percentages_df')

## lifestyle_sleep

In [122]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_lifestyle_sleep_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_lifestyle_sleep_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_lifestyle_sleep_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_lifestyle_sleep_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_lifestyle_sleep_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_lifestyle_sleep_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_lifestyle_sleep_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_lifestyle_sleep_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_lifestyle_sleep_percentages_df.csv')

In [158]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_lifestyle_sleep_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_lifestyle_sleep_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_lifestyle_sleep_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_lifestyle_sleep_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_lifestyle_sleep_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_lifestyle_sleep_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_lifestyle_sleep_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_lifestyle_sleep_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_lifestyle_sleep_percentages_df.csv')

In [159]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_lifestyle_sleep_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_lifestyle_sleep_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_lifestyle_sleep_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_lifestyle_sleep_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_lifestyle_sleep_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_lifestyle_sleep_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_lifestyle_sleep_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_lifestyle_sleep_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_lifestyle_sleep_percentages_df.csv')

In [123]:
transform_and_save_data(junior_data_business_analyst_df, 'junior', 'junior_Data-or-business-analyst_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_database_administrator_df, 'junior', 'junior_Database-administrator_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_developer_advocate_df, 'junior', 'junior_Developer-Advocate_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_Developer_back_end_df, 'junior', 'junior_Developer-back-end_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_Developer_front_end_df, 'junior', 'junior_Developer-front-end_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_Developer_full_stack_df, 'junior', 'junior_Developer-full-stack_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_Developer_mobile_df, 'junior', 'junior_Developer-mobile_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_DevOps_specialist_df, 'junior', 'junior_DevOps-specialist_lifestyle_sleep_percentages_df')
transform_and_save_data(junior_UX_UI_Designer_df, 'junior', 'junior_UX-UI-Designer_lifestyle_sleep_percentages_df')

In [160]:
transform_and_save_data(middle_data_business_analyst_df, 'middle', 'middle_Data-or-business-analyst_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_database_administrator_df, 'middle', 'middle_Database-administrator_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_developer_advocate_df, 'middle', 'middle_Developer-Advocate_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_Developer_back_end_df, 'middle', 'middle_Developer-back-end_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_Developer_front_end_df, 'middle', 'middle_Developer-front-end_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_Developer_full_stack_df, 'middle', 'middle_Developer-full-stack_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_Developer_mobile_df, 'middle', 'middle_Developer-mobile_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_DevOps_specialist_df, 'middle', 'middle_DevOps-specialist_lifestyle_sleep_percentages_df')
transform_and_save_data(middle_UX_UI_Designer_df, 'middle', 'middle_UX-UI-Designer_lifestyle_sleep_percentages_df')

In [161]:
transform_and_save_data(senior_data_business_analyst_df, 'senior', 'senior_Data-or-business-analyst_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_database_administrator_df, 'senior', 'senior_Database-administrator_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_developer_advocate_df, 'senior', 'senior_Developer-Advocate_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_Developer_back_end_df, 'senior', 'senior_Developer-back-end_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_Developer_front_end_df, 'senior', 'senior_Developer-front-end_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_Developer_full_stack_df, 'senior', 'senior_Developer-full-stack_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_Developer_mobile_df, 'senior', 'senior_Developer-mobile_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_DevOps_specialist_df, 'senior', 'senior_DevOps-specialist_lifestyle_sleep_percentages_df')
transform_and_save_data(senior_UX_UI_Designer_df, 'senior', 'senior_UX-UI-Designer_lifestyle_sleep_percentages_df')

## producive_to_job

In [124]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_productive_to_job_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_productive_to_job_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_productive_to_job_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_productive_to_job_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_productive_to_job_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_productive_to_job_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_productive_to_job_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_productive_to_job_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_productive_to_job_percentages_df.csv')

In [166]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_productive_to_job_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_productive_to_job_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_productive_to_job_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_productive_to_job_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_productive_to_job_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_productive_to_job_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_productive_to_job_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_productive_to_job_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_productive_to_job_percentages_df.csv')

In [169]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_productive_to_job_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_productive_to_job_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_productive_to_job_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_productive_to_job_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_productive_to_job_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_productive_to_job_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_productive_to_job_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_productive_to_job_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_productive_to_job_percentages_df.csv')

In [125]:
transform_and_save_data(junior_data_business_analyst_df, 'junior', 'junior_Data-or-business-analyst_productive_to_job_percentages_df')
transform_and_save_data(junior_database_administrator_df, 'junior', 'junior_Database-administrator_productive_to_job_percentages_df')
transform_and_save_data(junior_developer_advocate_df, 'junior', 'junior_Developer-Advocate_productive_to_job_percentages_df')
transform_and_save_data(junior_Developer_back_end_df, 'junior', 'junior_Developer-back-end_productive_to_job_percentages_df')
transform_and_save_data(junior_Developer_front_end_df, 'junior', 'junior_Developer-front-end_productive_to_job_percentages_df')
transform_and_save_data(junior_Developer_full_stack_df, 'junior', 'junior_Developer-full-stack_productive_to_job_percentages_df')
transform_and_save_data(junior_Developer_mobile_df, 'junior', 'junior_Developer-mobile_productive_to_job_percentages_df')
transform_and_save_data(junior_DevOps_specialist_df, 'junior', 'junior_DevOps-specialist_productive_to_job_percentages_df')
transform_and_save_data(junior_UX_UI_Designer_df, 'junior', 'junior_UX-UI-Designer_productive_to_job_percentages_df')

In [170]:
transform_and_save_data(middle_data_business_analyst_df, 'middle', 'middle_Data-or-business-analyst_productive_to_job_percentages_df')
transform_and_save_data(middle_database_administrator_df, 'middle', 'middle_Database-administrator_productive_to_job_percentages_df')
transform_and_save_data(middle_developer_advocate_df, 'middle', 'middle_Developer-Advocate_productive_to_job_percentages_df')
transform_and_save_data(middle_Developer_back_end_df, 'middle', 'middle_Developer-back-end_productive_to_job_percentages_df')
transform_and_save_data(middle_Developer_front_end_df, 'middle', 'middle_Developer-front-end_productive_to_job_percentages_df')
transform_and_save_data(middle_Developer_full_stack_df, 'middle', 'middle_Developer-full-stack_productive_to_job_percentages_df')
transform_and_save_data(middle_Developer_mobile_df, 'middle', 'middle_Developer-mobile_productive_to_job_percentages_df')
transform_and_save_data(middle_DevOps_specialist_df, 'middle', 'middle_DevOps-specialist_productive_to_job_percentages_df')
transform_and_save_data(middle_UX_UI_Designer_df, 'middle', 'middle_UX-UI-Designer_productive_to_job_percentages_df')

In [171]:
transform_and_save_data(senior_data_business_analyst_df, 'senior', 'senior_Data-or-business-analyst_productive_to_job_percentages_df')
transform_and_save_data(senior_database_administrator_df, 'senior', 'senior_Database-administrator_productive_to_job_percentages_df')
transform_and_save_data(senior_developer_advocate_df, 'senior', 'senior_Developer-Advocate_productive_to_job_percentages_df')
transform_and_save_data(senior_Developer_back_end_df, 'senior', 'senior_Developer-back-end_productive_to_job_percentages_df')
transform_and_save_data(senior_Developer_front_end_df, 'senior', 'senior_Developer-front-end_productive_to_job_percentages_df')
transform_and_save_data(senior_Developer_full_stack_df, 'senior', 'senior_Developer-full-stack_productive_to_job_percentages_df')
transform_and_save_data(senior_Developer_mobile_df, 'senior', 'senior_Developer-mobile_productive_to_job_percentages_df')
transform_and_save_data(senior_DevOps_specialist_df, 'senior', 'senior_DevOps-specialist_productive_to_job_percentages_df')
transform_and_save_data(senior_UX_UI_Designer_df, 'senior', 'senior_UX-UI-Designer_productive_to_job_percentages_df')

### learn_time

In [126]:
junior_data_business_analyst_df=pd.read_csv('../created_csv/junior/junior_Data-or-business-analyst_learn_time_percentages_df.csv')
junior_database_administrator_df=pd.read_csv('../created_csv/junior/junior_Database-administrator_learn_time_percentages_df.csv')
junior_developer_advocate_df=pd.read_csv('../created_csv/junior/junior_Developer-Advocate_learn_time_percentages_df.csv')
junior_Developer_back_end_df=pd.read_csv('../created_csv/junior/junior_Developer-back-end_learn_time_percentages_df.csv')
junior_Developer_front_end_df=pd.read_csv('../created_csv/junior/junior_Developer-front-end_learn_time_percentages_df.csv')
junior_Developer_full_stack_df=pd.read_csv('../created_csv/junior/junior_Developer-full-stack_learn_time_percentages_df.csv')
junior_Developer_mobile_df=pd.read_csv('../created_csv/junior/junior_Developer-mobile_learn_time_percentages_df.csv')
junior_DevOps_specialist_df=pd.read_csv('../created_csv/junior/junior_DevOps-specialist_learn_time_percentages_df.csv')
junior_UX_UI_Designer_df=pd.read_csv('../created_csv/junior/junior_UX-UI-Designer_learn_time_percentages_df.csv')

In [162]:
senior_data_business_analyst_df=pd.read_csv('../created_csv/senior/senior_Data-or-business-analyst_learn_time_percentages_df.csv')
senior_database_administrator_df=pd.read_csv('../created_csv/senior/senior_Database-administrator_learn_time_percentages_df.csv')
senior_developer_advocate_df=pd.read_csv('../created_csv/senior/senior_Developer-Advocate_learn_time_percentages_df.csv')
senior_Developer_back_end_df=pd.read_csv('../created_csv/senior/senior_Developer-back-end_learn_time_percentages_df.csv')
senior_Developer_front_end_df=pd.read_csv('../created_csv/senior/senior_Developer-front-end_learn_time_percentages_df.csv')
senior_Developer_full_stack_df=pd.read_csv('../created_csv/senior/senior_Developer-full-stack_learn_time_percentages_df.csv')
senior_Developer_mobile_df=pd.read_csv('../created_csv/senior/senior_Developer-mobile_learn_time_percentages_df.csv')
senior_DevOps_specialist_df=pd.read_csv('../created_csv/senior/senior_DevOps-specialist_learn_time_percentages_df.csv')
senior_UX_UI_Designer_df=pd.read_csv('../created_csv/senior/senior_UX-UI-Designer_learn_time_percentages_df.csv')

In [163]:
middle_data_business_analyst_df=pd.read_csv('../created_csv/middle/middle_Data-or-business-analyst_learn_time_percentages_df.csv')
middle_database_administrator_df=pd.read_csv('../created_csv/middle/middle_Database-administrator_learn_time_percentages_df.csv')
middle_developer_advocate_df=pd.read_csv('../created_csv/middle/middle_Developer-Advocate_learn_time_percentages_df.csv')
middle_Developer_back_end_df=pd.read_csv('../created_csv/middle/middle_Developer-back-end_learn_time_percentages_df.csv')
middle_Developer_front_end_df=pd.read_csv('../created_csv/middle/middle_Developer-front-end_learn_time_percentages_df.csv')
middle_Developer_full_stack_df=pd.read_csv('../created_csv/middle/middle_Developer-full-stack_learn_time_percentages_df.csv')
middle_Developer_mobile_df=pd.read_csv('../created_csv/middle/middle_Developer-mobile_learn_time_percentages_df.csv')
middle_DevOps_specialist_df=pd.read_csv('../created_csv/middle/middle_DevOps-specialist_learn_time_percentages_df.csv')
middle_UX_UI_Designer_df=pd.read_csv('../created_csv/middle/middle_UX-UI-Designer_learn_time_percentages_df.csv')

In [127]:
transform_and_save_data(junior_data_business_analyst_df, 'junior', 'junior_Data-or-business-analyst_learn_time_percentages_df')
transform_and_save_data(junior_database_administrator_df, 'junior', 'junior_Database-administrator_productive_to_job_percentages_df')
transform_and_save_data(junior_developer_advocate_df, 'junior', 'junior_Developer-Advocate_learn_time_percentages_df')
transform_and_save_data(junior_Developer_back_end_df, 'junior', 'junior_Developer-back-end_learn_time_percentages_df')
transform_and_save_data(junior_Developer_front_end_df, 'junior', 'junior_Developer-front-end_learn_time_percentages_df')
transform_and_save_data(junior_Developer_full_stack_df, 'junior', 'junior_Developer-full-stack_learn_time_percentages_df')
transform_and_save_data(junior_Developer_mobile_df, 'junior', 'junior_Developer-mobilelearn_timeb_percentages_df')
transform_and_save_data(junior_DevOps_specialist_df, 'junior', 'junior_DevOps-specialist_learn_time_percentages_df')
transform_and_save_data(junior_UX_UI_Designer_df, 'junior', 'junior_UX-UI-Designer_learn_time_percentages_df')

In [164]:
transform_and_save_data(middle_data_business_analyst_df, 'middle', 'middle_Data-or-business-analyst_learn_time_percentages_df')
transform_and_save_data(middle_database_administrator_df, 'middle', 'middle_Database-administrator_productive_to_job_percentages_df')
transform_and_save_data(middle_developer_advocate_df, 'middle', 'middle_Developer-Advocate_learn_time_percentages_df')
transform_and_save_data(middle_Developer_back_end_df, 'middle', 'middle_Developer-back-end_learn_time_percentages_df')
transform_and_save_data(middle_Developer_front_end_df, 'middle', 'middle_Developer-front-end_learn_time_percentages_df')
transform_and_save_data(middle_Developer_full_stack_df, 'middle', 'middle_Developer-full-stack_learn_time_percentages_df')
transform_and_save_data(middle_Developer_mobile_df, 'middle', 'middle_Developer-mobilelearn_timeb_percentages_df')
transform_and_save_data(middle_DevOps_specialist_df, 'middle', 'middle_DevOps-specialist_learn_time_percentages_df')
transform_and_save_data(middle_UX_UI_Designer_df, 'middle', 'middle_UX-UI-Designer_learn_time_percentages_df')

In [165]:
transform_and_save_data(senior_data_business_analyst_df, 'senior', 'seniorr_Data-or-business-analyst_learn_time_percentages_df')
transform_and_save_data(senior_database_administrator_df, 'senior', 'seniorr_Database-administrator_productive_to_job_percentages_df')
transform_and_save_data(senior_developer_advocate_df, 'senior', 'seniorr_Developer-Advocate_learn_time_percentages_df')
transform_and_save_data(senior_Developer_back_end_df, 'senior', 'seniorr_Developer-back-end_learn_time_percentages_df')
transform_and_save_data(senior_Developer_front_end_df, 'senior', 'seniorr_Developer-front-end_learn_time_percentages_df')
transform_and_save_data(senior_Developer_full_stack_df, 'senior', 'seniorr_Developer-full-stack_learn_time_percentages_df')
transform_and_save_data(senior_Developer_mobile_df, 'senior', 'seniorr_Developer-mobilelearn_timeb_percentages_df')
transform_and_save_data(senior_DevOps_specialist_df, 'senior', 'seniorr_DevOps-specialist_learn_time_percentages_df')
transform_and_save_data(senior_UX_UI_Designer_df, 'senior', 'seniorr_UX-UI-Designer_learn_time_percentages_df')